In [1]:
import pickle
import data.read_data as read
import data.transform_data as transform
from tqdm import tqdm
import torch as th
from math import ceil


In [2]:
model = pickle.load(open("saved/conv_3/model_150k_lowercase_12h_12m_35s_18_01_2019.p", "rb"))
vocab = pickle.load(open("saved/conv_3/vocab_150k_lowercase_12h_12m_35s_18_01_2019.p", "rb"))
count = pickle.load(open("saved/conv_3/count_150k_lowercase_12h_12m_35s_18_01_2019.p", "rb"))


In [3]:
df = read.load_train_csv("res/test.csv")


In [4]:
print(df.iloc[0:2, 0])


0    00001cee341fdb12
1    0000247867823ef7
Name: id, dtype: object


In [5]:
print(df.iloc[0:2, 1])


0    Yo bitch Ja Rule is more succesful then you'll...
1    == From RfC == \n\n The title is fine as it is...
Name: comment_text, dtype: object


In [7]:
print(df.iloc[-1, :])


id                                               ffffce3fb183ee80
comment_text    " \n :::Stop already. Your bullshit is not wel...
Name: 153163, dtype: object


In [8]:
idx_and_sent = transform.split_comment(df.iloc[:, :].values)


In [9]:
print(len(idx_and_sent))
print(idx_and_sent[0:2])


153164
[('00001cee341fdb12', array(['yo', 'bitch', 'ja', 'rule', 'is', 'more', 'succesful', 'then',
       'you', 'll', 'ever', 'be', 'whats', 'up', 'with', 'you', 'and',
       'hating', 'you', 'sad', 'mofuckas', 'i', 'should', 'bitch', 'slap',
       'ur', 'pethedic', 'white', 'faces', 'and', 'get', 'you', 'to',
       'kiss', 'my', 'ass', 'you', 'guys', 'sicken', 'me', 'ja', 'rule',
       'is', 'about', 'pride', 'in', 'da', 'music', 'man', 'dont', 'diss',
       'that', 'shit', 'on', 'him', 'and', 'nothin', 'is', 'wrong',
       'bein', 'like', 'tupac', 'he', 'was', 'a', 'brother', 'too',
       'fuckin', 'white', 'boys', 'get', 'things', 'right', 'next',
       'time'], dtype='<U9')), ('0000247867823ef7', array(['from', 'rfc', 'the', 'title', 'is', 'fine', 'as', 'it', 'is',
       'imo'], dtype='<U5'))]


In [10]:
idx, sentence = transform.pass_data_to_word_idx(idx_and_sent, vocab, count)
print(len(idx))


153164


In [11]:
padded_sentence = transform.pad_sentences(sentence, 1400, vocab[transform.padding])
print(padded_sentence.shape)


(153164, 1400)


In [12]:
batch_size = 32
nb_batch = ceil(padded_sentence.shape[0] / batch_size)
print(nb_batch)


4786


In [15]:
submission_name = "first_submission_conv_150k.csv"
file = open(submission_name, "w")
file.write("id,toxic,severe_toxic,obscene,threat,insult,identity_hate\n")


58

In [16]:
model.eval()
model.cuda()

for i in tqdm(range(nb_batch)):
    i_min = i * batch_size
    i_max = (i + 1) * batch_size
    i_max = i_max if i_max < padded_sentence.shape[0] else padded_sentence.shape[0]
    
    idx_batch = idx[i_min:i_max]
    sentence_batch = th.Tensor(padded_sentence[i_min:i_max]).long().cuda()
    
    res = model(sentence_batch).detach().cpu().numpy()
    
    to_write = ""
    for j in range(res.shape[0]):
        to_write += "{},{},{},{},{},{},{}\n".format(idx_batch[j], 
                                                    res[j, 0], res[j, 1], res[j, 2], res[j, 3], res[j, 4], res[j, 5])
    file.write(to_write)


  0%|          | 0/4787 [00:00<?, ?it/s]

  1%|          | 32/4787 [00:00<00:15, 314.12it/s]

  1%|▏         | 65/4787 [00:00<00:14, 317.82it/s]

  2%|▏         | 99/4787 [00:00<00:14, 321.75it/s]

  3%|▎         | 132/4787 [00:00<00:14, 321.51it/s]

...

 97%|█████████▋| 4662/4787 [00:13<00:00, 347.91it/s]

 98%|█████████▊| 4698/4787 [00:13<00:00, 349.32it/s]

 99%|█████████▉| 4733/4787 [00:13<00:00, 347.59it/s]

100%|█████████▉| 4769/4787 [00:13<00:00, 348.73it/s]

100%|██████████| 4787/4787 [00:13<00:00, 342.21it/s]

In [18]:
file.close()
